# 🧬 CPython Source Deep Dive: Object Memory Internals


This notebook walks through **CPython source code** segments relevant to:
- Object header layout
- Reference counting
- Garbage collection
- `__slots__` impact on memory

We refer to CPython's official source files such as `Include/object.h`, `Include/cpython/object.h`, `Objects/typeobject.c`, and `Include/longobject.h`.

---


## 📦 CPython: `PyObject_HEAD` in `object.h`

In [ ]:

# CPython C Macro (Include/object.h)

#define PyObject_HEAD                   \
    Py_ssize_t ob_refcnt;               \
    struct _typeobject *ob_type;

# This is the base layout of *every* object in CPython.
# - `ob_refcnt` tracks reference count
# - `ob_type` points to the type object (e.g., <class 'int'>)


## 🧱 PyVarObject (for objects with variable size like lists, strings, etc.)

In [ ]:

# CPython Macro (Include/object.h)

#define PyVarObject_HEAD_INIT(type, size) \
    PyObject_HEAD \
    Py_ssize_t ob_size;

# Used in lists, tuples, strings, etc.


## 🔁 Reference Counting Macros in CPython

In [ ]:

# Include/object.h

#define Py_INCREF(op) ((op)->ob_refcnt++)
#define Py_DECREF(op)                        \
    if (--(op)->ob_refcnt != 0)              \
        ;                                    \
    else                                     \
        _Py_Dealloc((PyObject *)(op))

# Every time a reference is assigned, Py_INCREF is used
# When objects go out of scope, Py_DECREF is used


## 🧳 `__slots__` in CPython: Avoiding __dict__

In [ ]:

# CPython's typeobject.c (simplified logic)

if (type->tp_dictoffset == 0 && !type->tp_weaklistoffset) {
    // __dict__ and __weakref__ are not created
    // Memory savings!
}

# __slots__ affect type layout by preventing allocation of __dict__ and __weakref__


## 🔍 Debugging Memory with CPython Hooks

In [ ]:

# Python's built-in debugging API (Python-level)

import sys

def track(obj):
    print(f"Type: {type(obj)}, Refcount: {sys.getrefcount(obj)}, Size: {sys.getsizeof(obj)}")

x = [1, 2, 3]
track(x)


## 🧭 CPython Source References


- [`Include/object.h`](https://github.com/python/cpython/blob/main/Include/object.h) – Core object macros  
- [`Include/cpython/object.h`](https://github.com/python/cpython/blob/main/Include/cpython/object.h) – Refcount, internals  
- [`Objects/typeobject.c`](https://github.com/python/cpython/blob/main/Objects/typeobject.c) – Type creation logic  
- [`Include/longobject.h`](https://github.com/python/cpython/blob/main/Include/longobject.h) – Integer interning  
- [`Modules/gcmodule.c`](https://github.com/python/cpython/blob/main/Modules/gcmodule.c) – GC system
